In [ ]:
criteria_list = [o3d.t.pipelines.registration.ICPConvergenceCriteria(relative_fitness=1e-4, 
                                                                     relative_rmse=1e-4, 
                                                                     max_iteration=100),
                 o3d.t.pipelines.registration.ICPConvergenceCriteria(relative_fitness=1e-5, 
                                                                     relative_rmse=1e-5, 
                                                                     max_iteration=60),
                 o3d.t.pipelines.registration.ICPConvergenceCriteria(relative_fitness=1e-6, 
                                                                     relative_rmse=1e-6, 
                                                                     max_iteration=30),]

In [ ]:
result = o3d.t.pipelines.registration.registration_multi_scale_icp(tsdf_all_cameras[4], 
                                                                   tsdf_all_cameras[4], 
                                                                   o3d.utility.DoubleVector([voxel_size*4, 
                                                                                             voxel_size*2, 
                                                                                             voxel_size]), 
                                                                   criteria_list,
                                                                   o3d.utility.DoubleVector([0.3, 0.15, 0.07]),
                                                                   o3d.core.Tensor(initial_transforms[4]))

In [ ]:
result.transformation.cuda()


In [ ]:
volume = o3d.t.geometry.TSDFVoxelGrid(
    map_attrs_to_dtypes={
        'tsdf': o3d.core.Dtype.Float32,
        'weight': o3d.core.Dtype.UInt16,
        'color': o3d.core.Dtype.UInt16
    },
    voxel_size=0.02,
    sdf_trunc=0.005,
    block_resolution=16,
    block_count=50000,
    device=device,
)

In [ ]:

extrinsic_gpu1 = result.transformation.cuda()
intrinsic1 = o3d.camera.PinholeCameraIntrinsic(width=camera_info_all_cameras[0]["width"],
                                              height=camera_info_all_cameras[0]["height"],
                                              fx=camera_info_all_cameras[0]["fx"], fy=camera_info_all_cameras[0]["fy"],
                                              cx=camera_info_all_cameras[0]["cx"], cy=camera_info_all_cameras[0]["cy"])
intrinsic_gpu1 = o3d.core.Tensor(intrinsic1.intrinsic_matrix, o3d.core.Dtype.Float32, device)
​
extrinsic_gpu5 = extrinsic_gpu = o3d.core.Tensor(np.eye(4), o3d.core.Dtype.Float32, device)
​
intrinsic5 = o3d.camera.PinholeCameraIntrinsic(width=camera_info_all_cameras[4]["width"],
                                              height=camera_info_all_cameras[4]["height"],
                                              fx=camera_info_all_cameras[4]["fx"], fy=camera_info_all_cameras[4]["fy"],
                                              cx=camera_info_all_cameras[4]["cx"], cy=camera_info_all_cameras[4]["cy"])
intrinsic_gpu5 = o3d.core.Tensor(intrinsic5.intrinsic_matrix, o3d.core.Dtype.Float32, device)
​
volume.integrate(depth=masked_depth_all_cameras[0][1],
                         color=color_frames_all_cameras[0][1],
                         intrinsics=intrinsic_gpu1,
                         extrinsics=extrinsic_gpu1,
                         depth_scale=1000.0,
                         depth_max=1.2,
                         )
volume.integrate(depth=masked_depth_all_cameras[4][1],
                 color=color_frames_all_cameras[4][1],
                 intrinsics=intrinsic_gpu5,
                 extrinsics=extrinsic_gpu5,
                 depth_scale=1000.0,
                         depth_max=1.2,
                         )